# pyNBL: Basketball Statistic System for Australian NBL

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ssardina/pynbl/HEAD)

This notebook **incrementally** builds a set of stat tables from NBL Basketball Games:

1. A table of games played, with team names, points, venue, etc.
2. A stat table of _stint lineups_ (advance) statistics for each game and each team. A **stint** is a lineup of players who play together in different interval periods across the game. This table will contain the stints for each team from the play-by-play data and compute various statistics for those stints.

Tables will be saved in CSV and Excel formats as well as in [Pickle format](https://docs.python.org/3/library/pickle.html) for later recovery as Panda DataFrames.


The data comes as a raw JSON file using the game id (e.g., `2087737`):

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [1]:
# Let's first load all required packages...
import os
from pathlib import Path
import pandas as pd
import numpy as np
import dtale

from config import *
import bball_stats
import tools


# Set folder with data files and Pickle tables saved on disk
DATA_DIR='data-22_23/'
FILES = dict()
FILES['stint_stats'] = Path(DATA_DIR, "stint_stats_df").with_suffix('.pkl')
FILES['stints'] = Path(DATA_DIR, "stints_df").with_suffix('.pkl')
FILES['games'] = Path(DATA_DIR, "games_df").with_suffix('.pkl')
FILES['players'] = Path(DATA_DIR, "players_df").with_suffix('.pkl')

## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [2]:
# Games to be computed  - Format: (game id, round number)

games_20_21 = [('1816803', 1), ('1816804', 1), ('1816920', 1), ('1816802', 1), ('1816801', 1), ('1817368', 2), ('1817370', 2), ('1817371', 2), ('1817372', 2), ('1817369', 2), ('1817367', 2), ('1843512', 3), ('1843514', 3), ('1843517', 3), ('1843515', 3), ('1843516', 3), ('1843513', 3), ('1843511', 3), ('1846331', 4), ('1846335', 4), ('1846338', 4), ('1846333', 4), ('1846336', 4), ('1846334', 4), ('1848323', 5), ('1848327', 5), ('1848322', 5), ('1848326', 5), ('1848324', 5), ('1848329', 5), ('1848328', 5), ('1848990', 6), ('1848985', 6), ('1848989', 6), ('1848986', 6), ('1848984', 6), ('1848988', 6), ('1848991', 6), ('1848987', 6), ('1848995', 7), ('1848999', 7), ('1848992', 7), ('1848996', 7), ('1848994', 7), ('1848997', 7), ('1848993', 7), ('1848998', 7), ('1849003', 8), ('1849007', 8), ('1849001', 8), ('1849002', 8), ('1849008', 8), ('1849009', 8), ('1849006', 8), ('1849005', 8), ('1849004', 8), ('1849000', 8), ('1849010', 9), ('1849019', 9), ('1849017', 9), ('1849012', 9), ('1849018', 9), ('1849015', 9), ('1849016', 9), ('1849013', 9), ('1849014', 9), ('1849011', 9), ('1859485', 10), ('1859486', 10), ('1859487', 10), ('1859484', 10), ('1859482', 10), ('1859483', 10), ('1859481', 10), ('1862515', 11), ('1862514', 11), ('1862517', 11), ('1862516', 11), ('1862518', 11), ('1862519', 11), ('1862520', 11), ('1866357', 12), ('1866351', 12), ('1866355', 12), ('1866356', 12), ('1866354', 12), ('1866350', 12), ('1866352', 12), ('1866363', 13), ('1866367', 13), ('1866360', 13), ('1866362', 13), ('1866364', 13), ('1866366', 13), ('1866365', 13), ('1866361', 13), ('1866368', 13), ('1869882', 14), ('1869881', 14), ('1866370', 14), ('1866374', 14), ('1866373', 14), ('1866371', 14), ('1866369', 14), ('1866372', 14), ('1869890', 15), ('1869886', 15), ('1869889', 15), ('1869885', 15), ('1869884', 15), ('1869883', 15), ('1869887', 15), ('1875311', 16), ('1875307', 16), ('1875304', 16), ('1875310', 16), ('1875305', 16), ('1875312', 16), ('1875306', 16), ('1875309', 16), ('1875313', 17), ('1875316', 17), ('1875314', 17), ('1875315', 17), ('1880553', 18), ('1875321', 18), ('1875324', 18), ('1875325', 18), ('1875319', 18), ('1875317', 18), ('1875322', 18), ('1875318', 18), ('1875323', 18), ('1875320', 18), ('1875328', 19), ('1875333', 19), ('1875327', 19), ('1875329', 19), ('1875330', 19), ('1875336', 19), ('1875326', 19), ('1875335', 19), ('1875332', 19), ('1875331', 19), ('1875334', 19), ('1875339', 20), ('1875340', 20), ('1875338', 20), ('1875344', 20), ('1875342', 20), ('1875341', 20), ('1875343', 20), ('1875337', 20), ('1875348', 21), ('1875350', 21), ('1880554', 21), ('1875349', 21), ('1875352', 21), ('1875347', 21), ('1875346', 21), ('1875345', 21), ('1875351', 21), ('1889799', 100), ('1889797', 100), ('1891192', 100), ('1891188', 100), ('1889798', 100), ('1889796', 100), ('1891805', 101), ('1891806', 101), ('1894782', 101)]

games_21_22 = [('2004608', 1), ('1976448', 1), ('1976452', 1), ('1976454', 1), ('1976446', 1), ('1976447', 1), ('1976451', 2), ('1976455', 2), ('1976456', 2), ('1976453', 2), ('1976458', 2), ('1976449', 2), ('2004609', 2), ('1976461', 3), ('1976463', 3), ('1976459', 3), ('1976460', 3), ('1976462', 3), ('1976464', 3), ('1976457', 3), ('1976465', 3), ('2004610', 3), ('1976468', 4), ('1976469', 4), ('1976474', 5), ('1976473', 5), ('1976482', 6), ('2036215', 7), ('2031330', 7), ('2031332', 7), ('2031333', 7), ('2031329', 7), ('2031334', 7), ('2046695', 8), ('2031341', 8), ('2031335', 8), ('2046696', 8), ('2031338', 8), ('2031340', 8), ('2031337', 8), ('2031336', 8), ('2046697', 9), ('2031345', 9), ('2031343', 9), ('2031347', 9), ('2031342', 9), ('2031344', 9), ('2031346', 9), ('2046702', 10), ('2046704', 10), ('2046701', 10), ('2046703', 10), ('2046698', 10), ('2046700', 10), ('2046706', 10), ('2046711', 11), ('2051763', 11), ('2046712', 11), ('2046713', 11), ('2046709', 11), ('2046710', 11), ('2046707', 11), ('2053812', 12), ('2053816', 12), ('2053811', 12), ('2053815', 12), ('2053813', 12), ('2053814', 12), ('2053817', 12), ('2053818', 13), ('2053819', 13), ('2053822', 13), ('2053823', 13), ('2053821', 13), ('2053825', 13), ('2053820', 13), ('2053824', 13), ('2056457', 14), ('2056461', 14), ('2056462', 14), ('2056460', 14), ('2056458', 14), ('2056455', 14), ('2056454', 14), ('2056467', 15), ('2056472', 15), ('2056473', 15), ('2056466', 15), ('2056464', 15), ('2056469', 15), ('2056463', 15), ('2056471', 15), ('2065657', 16), ('2065653', 16), ('2065656', 16), ('2065655', 16), ('2065654', 16), ('2065659', 16), ('2065658', 16), ('2069172', 17), ('2069165', 17), ('2069171', 17), ('2069170', 17), ('2069169', 17), ('2069166', 17), ('2069168', 17), ('2069167', 17), ('2069175', 18), ('2069184', 18), ('2069181', 18), ('2069177', 18), ('2069183', 18), ('2069179', 18), ('2069186', 18), ('2069187', 18), ('2069202', 19), ('2069203', 19), ('2069196', 19), ('2069204', 19), ('2069205', 19), ('2069191', 19), ('2069192', 19), ('2069199', 19), ('2069194', 19), ('2069195', 20), ('2069189', 20), ('2069193', 20), ('2069197', 20), ('2069201', 20), ('2069200', 20), ('2069190', 20), ('2069198', 20), ('2069176', 21), ('2069185', 21), ('2069180', 21), ('2069178', 21), ('2069174', 21), ('2069173', 21), ('2069188', 21), ('2069182', 21), ('2087734', 100), ('2087735', 100), ('2087733', 100), ('2087737', 100), ('2087736', 100), ('2090350', 101), ('2090352', 101), ('2090351', 101)]


# We build each round incrementally
games_22_23 = []
games_22_23.extend([('2122059', 0), ('2120077', 0), ('2120053', 0), ('2120079', 0), ('2120054', 0), ('2122060', 0), ('2141127', 0), ('2116579', 0), ('2135117', 0), ('2120050', 0), ('2141126', 0), ('2120049', 0), ('2135116', 0), ('2120048', 0), ('2120078', 0), ('2120056', 0), ('2134935', 0), ('2120055', 0), ('2120080', 0), ('2120058', 0), ('2116576', 0), ('2120052', 0), ('2120057', 0), ('2124207', 0), ('2120051', 0)])

DATA_DIR, games = 'data-20_21/', games_20_21
# DATA_DIR, games = 'data-21_22/', games_21_22
# DATA_DIR, games = 'data-22_23/', games_22_23

print("Games to scrape:", games)
print("Folder to be used:", DATA_DIR)

# Set to True to re-compute from scratch all tables
reload = False

Games to scrape: [('1816803', 1), ('1816804', 1), ('1816920', 1), ('1816802', 1), ('1816801', 1), ('1817368', 2), ('1817370', 2), ('1817371', 2), ('1817372', 2), ('1817369', 2), ('1817367', 2), ('1843512', 3), ('1843514', 3), ('1843517', 3), ('1843515', 3), ('1843516', 3), ('1843513', 3), ('1843511', 3), ('1846331', 4), ('1846335', 4), ('1846338', 4), ('1846333', 4), ('1846336', 4), ('1846334', 4), ('1848323', 5), ('1848327', 5), ('1848322', 5), ('1848326', 5), ('1848324', 5), ('1848329', 5), ('1848328', 5), ('1848990', 6), ('1848985', 6), ('1848989', 6), ('1848986', 6), ('1848984', 6), ('1848988', 6), ('1848991', 6), ('1848987', 6), ('1848995', 7), ('1848999', 7), ('1848992', 7), ('1848996', 7), ('1848994', 7), ('1848997', 7), ('1848993', 7), ('1848998', 7), ('1849003', 8), ('1849007', 8), ('1849001', 8), ('1849002', 8), ('1849008', 8), ('1849009', 8), ('1849006', 8), ('1849005', 8), ('1849004', 8), ('1849000', 8), ('1849010', 9), ('1849019', 9), ('1849017', 9), ('1849012', 9), ('1849

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, computes stats and game info, and adds them to the initial tables of stats and games.

We start by loading all saved previous games, if any, as we want to append to that database (and we don't want to recompute them).

In [3]:
# Load tables from saved files (if any)
saved_stint_stats_df = None
saved_stints_df = None
saved_games_df = None
saved_players_df = None
existing_games = []

if not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading recorded dataframes from files")
    try:
        saved_stint_stats_df = pd.read_pickle(FILES['stint_stats'])
        saved_stints_df = pd.read_pickle(FILES['stints'])
        saved_games_df = pd.read_pickle(FILES['games'])
        saved_players_df = pd.read_pickle(FILES['players'])
        # collect game ids of all games recovered from file
        existing_games = saved_games_df.game_id.unique()
    except FileNotFoundError as e:
        print("Error loading Pickle files: ", e)
        saved_stint_stats_df = None
        saved_stints_df = None
        saved_games_df = None
        saved_players_df = None
        existing_games = []
else:
    existing_games = []

print(f"Recovered {len(existing_games)} games: ", existing_games)

# saved_stats_df['lineup'].apply(lambda x: len(x) > 5)
# saved_stats_df.loc[5,'lineup']
# saved_stats_df.loc[5]

# saved_stint_stats_df.sample(3)
# saved_games_df.sample(3)

Recovered 0 games:  []


It is now time to process games to extract:

1. Table of **games**.
2. Table of **players** who played in each game with their stats, for each team.
3. Table of **stints** in each game for each team.
4. Table of **stint stats** in each game for each team.

In [4]:
# collect here set of stat dfs and game info, one per game
#   then, we will put them together into different dataframes
from urllib.error import HTTPError


stint_stats_dfs = []
stints_dfs = []
players_dfs = []
games_data = []

# Build data for each game, we'll put them together after....
for game in games:
    # get game_id and round no (if available)
    if isinstance(game, tuple):
        game_id, round_no = game
    else:
        game_id = game
        round_no = np.nan # no round info available

    # don't scrape game data if already loaded from file, skip it
    if game_id in existing_games:
        print(f"Game {game_id} was already saved on file; no scrapping...")
        continue

    ##################################################################
    # !!! MAIN STEP: scrape and compute the actual stats for the game
    ##################################################################
    print(f"Computing game {game_id}...")

    # 1. Read game JSON file
    try:
        game_json = tools.get_json_data(game_id)
    except HTTPError as e:
        print(f"Game {game_id} JSON data not available yet: ", e)
        continue


    result = bball_stats.build_game_stints_stats_df(game_json, game_id)
    game_stint_stats_df = result['stint_stats_df']   #  this is basically what we care, the stint stats
    game_stints_df = result['stints_df']
    game_team1, game_team2 = result['teams']

    # Add the game id column to game tables
    game_stint_stats_df.insert(0, 'game_id', game_id)
    game_stints_df.insert(0, 'game_id', game_id)

    # Extract players in the game
    players_df = bball_stats.get_players_stats(game_json)
    players_df.insert(0, 'game_id', game_id)

    # Add tables to collected set of tables, one per game
    stint_stats_dfs.append(game_stint_stats_df)
    stints_dfs.append(game_stints_df)
    players_dfs.append(players_df)

    # Next build the record for the game dataframe
    # first, extract date of game from HTML page
    try:
        game_info = tools.get_game_info(game_id)
    except:
        game_info = { "venue" : np.nan, "date": np.nan}
    print(f"\t .... done: {game_team1[0]} ({game_team1[1]}) vs {game_team2[0]} ({game_team2[1]}) on {game_info['date']}")

    games_data.append({"game_id": game_id,
                        "date" : game_info['date'],
                        "round": round_no,
                        "team1": game_team1[0],
                        "team2": game_team2[0],
                        "s1": game_team1[1],
                        "s2": game_team2[1],
                        "winner": 1 if game_team1[1] > game_team2[1] else 2,
                        "venue" : game_info["venue"]}
                      )


#################################
# All games have been processed, now put all dfs together
#################################
if len(games_data) == 0:
    raise SystemExit("No games!")

# First, build a dataframe with all the game data collected
games_df = pd.DataFrame(games_data) if saved_games_df is None else pd.concat([saved_games_df, pd.DataFrame(games_data)])
games_df.reset_index(inplace=True, drop=True)

# Build players dataframe
players_df = pd.concat(players_dfs + ([saved_players_df] if saved_players_df is not None else []))
players_df.reset_index(inplace=True, drop=True)

# Build stint stats dataframe
stint_stats_df = pd.concat(stint_stats_dfs + ([saved_stint_stats_df] if saved_stint_stats_df is not None else []))
stint_stats_df.reset_index(inplace=True, drop=True)

# Build stints dataframe
stints_df = pd.concat(stints_dfs + ([saved_stints_df] if saved_stints_df is not None else []))
stints_df.reset_index(inplace=True, drop=True)

print("Number of total games collected: ", games_df.shape[0])
print("Number of NEW collected: ", games_df.shape[0] - len(existing_games))
print("Number of FAILED games (not yet played): ", len(games) - games_df.shape[0])

stint_stats_df.sample(2)
stints_df.sample(2)
games_df.sample(5)
players_df.sample(5)

Computing game 2004608...
	 .... done: Perth Wildcats (90) vs Cairns Taipans (67) on 2021-12-05 00:00:00
Computing game 1976448...
	 .... done: South East Melbourne Phoenix (89) vs New Zealand Breakers (65) on 2021-12-04 00:00:00
Computing game 1976452...
	 .... done: Adelaide 36ers (71) vs Illawarra Hawks (81) on 2021-12-05 00:00:00
Computing game 1976454...
	 .... done: Sydney Kings (79) vs Melbourne United (74) on 2021-12-05 00:00:00
Computing game 1976446...


2022-09-18 10:23:45 WARNING Sub team 1 @ 00:00:17.800000 in period 5: incoming players already in court: {'Matt Kenyon'}
2022-09-18 10:23:45 WARNING Sub team 1 @ 00:00:17.800000 in period 5: outcoming players not in court: {'Jack McVeigh'}


	 .... done: Tasmania JackJumpers (83) vs Brisbane Bullets (74) on 2021-12-03 00:00:00
Computing game 1976447...


2022-09-18 10:23:49 WARNING Sub team 1 @ 00:02:26 in period 3: incoming players already in court: {'Jack Purchase'}


	 .... done: Perth Wildcats (85) vs Adelaide 36ers (73) on 2021-12-03 00:00:00
Computing game 1976451...
	 .... done: Cairns Taipans (69) vs Tasmania JackJumpers (62) on 2021-12-11 00:00:00
Computing game 1976455...
	 .... done: Adelaide 36ers (98) vs New Zealand Breakers (85) on 2021-12-12 00:00:00
Computing game 1976456...
	 .... done: Tasmania JackJumpers (80) vs Adelaide 36ers (83) on 2021-12-09 00:00:00
Computing game 1976453...
	 .... done: Melbourne United (86) vs South East Melbourne Phoenix (94) on 2021-12-12 00:00:00
Computing game 1976458...
	 .... done: South East Melbourne Phoenix (95) vs New Zealand Breakers (88) on 2021-12-10 00:00:00
Computing game 1976449...
	 .... done: Sydney Kings (84) vs Illawarra Hawks (92) on 2021-12-11 00:00:00
Computing game 2004609...
	 .... done: Perth Wildcats (94) vs Brisbane Bullets (97) on 2021-12-12 00:00:00
Computing game 1976461...
	 .... done: Sydney Kings (84) vs South East Melbourne Phoenix (73) on 2021-12-18 00:00:00
Computing game

...

2022-09-18 10:36:09 WARNING Sub team 1 @ 00:00:02.900000 in period 4: incoming players already in court: {'Angus Glover'}


	 .... done: Sydney Kings (84) vs Illawarra Hawks (87) on 2022-04-24 00:00:00
Computing game 2069173...
	 .... done: Perth Wildcats (100) vs South East Melbourne Phoenix (102) on 2022-04-24 00:00:00
Computing game 2069188...
	 .... done: Cairns Taipans (77) vs Sydney Kings (87) on 2022-04-21 00:00:00
Computing game 2069182...
	 .... done: Perth Wildcats (77) vs Illawarra Hawks (82) on 2022-04-22 00:00:00
Computing game 2087734...
	 .... done: Illawarra Hawks (79) vs Sydney Kings (89) on 2022-04-29 00:00:00
Computing game 2087735...
	 .... done: Tasmania JackJumpers (79) vs Melbourne United (72) on 2022-04-30 00:00:00
Computing game 2087733...
	 .... done: Melbourne United (74) vs Tasmania JackJumpers (63) on 2022-04-28 00:00:00
Computing game 2087737...
	 .... done: Melbourne United (73) vs Tasmania JackJumpers (76) on 2022-05-02 00:00:00
Computing game 2087736...
	 .... done: Sydney Kings (99) vs Illawarra Hawks (87) on 2022-05-01 00:00:00
Computing game 2090350...
	 .... done: Sydney

2022-09-18 10:37:44 WARNING Sub team 2 @ 00:00:25.200000 in period 2: incoming players already in court: {'Ian Clark'}


	 .... done: Tasmania JackJumpers (86) vs Sydney Kings (90) on 2022-05-08 00:00:00
Number of total games collected:  148
Number of NEW collected:  148
Number of FAILED games (not yet played):  0


,game_id,tno,player,shirtNumber,sMinutes,sFieldGoalsMade,sFieldGoalsAttempted,sFieldGoalsPercentage,sThreePointersMade,sThreePointersAttempted,...,starter,photoT,photoS,playingPosition,name,comp.sMinutesAverage,comp.sPointsAverage,comp.sReboundsTotalAverage,comp.sAssistsAverage,captain
3022,2090351,1,Clint Steindl,35,00:19:20,2,9,22,2,8,...,0,http://img.wh.sportingpulseinternational.com/7...,http://img.wh.sportingpulseinternational.com/7...,F,C. Steindl,20:03,9.06,2.45,0.91,True
2336,2069186,2,Ben Ayre,10,00:03:33,1,4,25,1,2,...,0,NaN,NaN,PG,B. Ayre,4:58,2.00,1.00,0.50,False
1453,2053819,2,Daniel Johnson,21,00:28:52,4,14,28,2,7,...,1,http://img.wh.sportingpulseinternational.com/c...,http://img.wh.sportingpulseinternational.com/c...,F,D. Johnson,31:23,14.54,6.77,2.00,False
1548,2053820,2,MiKyle McIntosh,0,00:23:35,9,11,81,1,2,...,0,http://img.wh.sportingpulseinternational.com/f...,http://img.wh.sportingpulseinternational.com/f...,F,M. Mcintosh,16:18,7.00,3.40,0.67,False
1030,2046704,1,Mason Peatling,22,00:23:03,1,3,33,0,0,...,0,http://img.wh.sportingpulseinternational.com/9...,http://img.wh.sportingpulseinternational.com/9...,F,M. Peatling,16:15,5.55,4.36,0.36,False


If we want we can do some sanity checks, before saving to disk:

In [ ]:
dtale.show(games_df)
dtale.show(stint_stats_df)
# dtale.show(stints_df)
# dtale.show(players_df)

In [6]:
import random
print("The shape of stats_df is:", stint_stats_df.shape)
stats_cols = list(stint_stats_df.columns[4:-49])
print("Stats cols:", stats_cols)

# build columns we want to show
cols = ['game_id' , 'tno', 'team', 'stint']
rnd_cols = random.sample(stats_cols, 8)
rnd_cols.extend([f"{x}_opp" for x in rnd_cols])
cols.extend(rnd_cols)

# show some sample of stats computed
stint_stats_df[cols].sample(5)

The shape of stats_df is: (4987, 99)
Stats cols: ['poss', 'ortg', 'drtg', 'nrtg', 'fga', 'fgm', 'fgp', 'pts', 'patra', 'patrm', 'patrp', '3pt_fga', '3pt_fgm', '3pt_fgp', '2pt_fga', '2pt_fgm', '2pt_fgp', 'fta', 'ftm', 'ftp', 'tsp', 'ast', 'astr', 'fgm_astp', 'stl', 'stlr', 'blk', 'blkr', 'tov', 'tovr', 'reb', 'dreb', 'drebc', 'drebp', 'oreb', 'odrec', 'orebp', 'trb', 'trbr', 'tov_bh', 'tov_bp', 'tov_ofoul', 'tov_3sec', 'tov_8sec', 'tov_24sec', 'opp_fga_blocked']


,game_id,tno,team,stint,ftp,fta,fgp,fga,stl,trb,odrec,tov_24sec,ftp_opp,fta_opp,fgp_opp,fga_opp,stl_opp,trb_opp,odrec_opp,tov_24sec_opp
4228,2069189,1,Melbourne United,21,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.0,1.0,0.0
1157,2031341,2,Cairns Taipans,13,0.0,0.0,0.00,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.00,2.0,0.0,1.0,1.0,0.0
2854,2056472,1,Cairns Taipans,15,100.0,3.0,50.00,10.0,2.0,7.0,5.0,0.0,100.0,1.0,28.57,7.0,0.0,3.0,5.0,1.0
2125,2053816,2,New Zealand Breakers,14,50.0,2.0,44.44,9.0,2.0,4.0,6.0,0.0,0.0,2.0,37.50,8.0,0.0,8.0,6.0,0.0
1946,2046713,2,Brisbane Bullets,10,0.0,0.0,50.00,2.0,0.0,2.0,1.0,0.0,0.0,1.0,50.00,2.0,0.0,1.0,2.0,0.0


Sanity check that `(ortg, drtg)` (offensive/defensive rate goal) should mirror `(drtg_opp, ortg_opp)` (opponent offensive/defensive rate goal)):

In [5]:
# (ortg, drtg) should mirror (drtg_opp, ortg_opp)
stint_stats_df.iloc[4][['game_id' , 'team', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

game_id            2004608
team        Perth Wildcats
poss                   4.0
ortg                  50.0
drtg                  50.0
poss_opp               4.0
ortg_opp              50.0
drtg_opp              50.0
Name: 4, dtype: object

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [7]:
import datetime
import os
import shutil
from pathlib import Path

# make a backup of existing tables on files
for pkl_file in FILES.values():
    for ext in ['.csv', '.xlsx', '.pkl']:
        file = Path(pkl_file).with_suffix(ext)
        if os.path.exists(file):
            # print("Backup file", file)
            shutil.copy(file, str(file) + ".bak")

# dump stint stats dataframe
stint_stats_df.to_pickle(Path(DATA_DIR, "stint_stats_df").with_suffix(".pkl"))
stint_stats_df.to_csv(Path(DATA_DIR, "stint_stats_df").with_suffix(".csv"), index=False)
stint_stats_df.to_excel(Path(DATA_DIR, "stint_stats_df").with_suffix(".xlsx"), index=False)

# dump stint stats dataframe
stints_df.to_pickle(Path(DATA_DIR, "stints_df").with_suffix(".pkl"))
stints_df.to_csv(Path(DATA_DIR, "stints_df").with_suffix(".csv"), index=False)
stints_df.to_excel(Path(DATA_DIR, "stints_df").with_suffix(".xlsx"), index=False)

# dump game dataframe
games_df.to_pickle(Path(DATA_DIR, "games_df").with_suffix(".pkl"))
games_df.to_csv(Path(DATA_DIR, "games_df").with_suffix(".csv"), index=False)
games_df.to_excel(Path(DATA_DIR, "games_df").with_suffix(".xlsx"), index=False)

# dump players dataframe
players_df.to_pickle(Path(DATA_DIR, "players_df").with_suffix(".pkl"))
players_df.to_csv(Path(DATA_DIR, "players_df").with_suffix(".csv"), index=False)
players_df.to_excel(Path(DATA_DIR, "players_df").with_suffix(".xlsx"), index=False)

now = datetime.datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

print(f"Finished saving in {DATA_DIR} @ {date_time}")

Finished saving in data-21_22/ @ 09/18/2022, 10:43:59


## 4. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [ ]:
dtale.show(stint_stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## 5. Some checks...

Check if a stint lineup has more than 5 players! It could happen:

1. Game 2031329, player H. Besson comes out (wrongly?) at 3rd period min 10:00 but he keeps playing and then goes out again at 7:33.

In [ ]:
stint_stats_df.shape
mask = stint_stats_df['lineup'].apply(lambda x: len(x) != 5)
stint_stats_df[mask]

# stats_df.iloc[941][['game_id', 'lineup']]